In [1]:
import os
os.chdir('/home/149/ab8992/cosima_regional/development/regional-mom6')
# from recipes import surfacespeed
from importlib import reload
# import ttidelib as tt
# from ttidelib import logmsg
import shutil
import subprocess
import dask
import xarray as xr
from matplotlib import pyplot as plt
from dask.distributed import Client, default_client
from pathlib import Path
import sys
# sys.path.append('/home/149/ab8992/cosima_regional/development/regional-mom6/regional_mom6')
import regional_mom6 as rm
try:
    # Try to get the existing Dask client
    client = default_client()
except ValueError:
    # If there's no existing client, create a new one
    client = Client()

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/40029/status,
Dashboard: /proxy/40029/status,Workers: 7
Total threads: 28,Total memory: 112.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42743,Workers: 7
Dashboard: /proxy/40029/status,Total threads: 28
Started: Just now,Total memory: 112.00 GiB
Comm: tcp://127.0.0.1:34283,Total threads: 4
Dashboard: /proxy/37521/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:44659,


In [2]:
cd /home/149/ab8992/tasman-tides

/home/149/ab8992/tasman-tides


In [3]:
import ttidelib as tt

def startdask():
    try:
    # Try to get the existing Dask client
        client = default_client()
    except ValueError:
        # If there's no existing client, create a new one
        client = Client()
home = Path("/home/149/ab8992/tasman-tides")
gdata = Path("/g/data/nm03/ab8992")

def save_vorticity(experiment,outputs,recompute = False):
    """
    Save the relative vorticity for the given experiment and outputs
    """
    basepath = gdata / "postprocessed" / experiment / "vorticity"
    startdask()


    rawdata = tt.collect_data(
        experiment,
        outputs=outputs,
        rawdata = ["u","v"],
        chunks = {"time": -1,"xb":-1,"zl":10}
        )

    vorticity_topdown = tt.calculate_vorticity(rawdata).coarsen(time = 149,boundary = "trim").mean().drop("lat").drop("lon").rename("vorticity").isel(zl = 2)
    vorticity_transect = tt.calculate_vorticity(rawdata).coarsen(time = 149,boundary = "trim").mean().drop("lat").drop("lon").rename("vorticity").sel(yb = 0,method = "nearest")

    tt.save_ppdata(vorticity_transect,vorticity_topdown,basepath,recompute=recompute)

    return 

I think there are outputs now that have messed up y chunking from where I redit it.

In [10]:
import glob
def check(expt):
    if "40" in expt:
        no = 34
    else:
        no = 17

    dirs = os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}")
    vars = ["ahh","v","u","e","rho"]
    # tell me how many files in each directory
    # print(dirs)
    for dir in dirs:
        if dir[0] != "o":
            continue

        bad = False
        for var in vars:
            # print(len(os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var)))
            
            if os.path.exists(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var) and len(glob.glob(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var + "/*.nc")) != no:
                
                if bad == False:
                    print(dir + ":",end = "\t")
                bad = True
                print(var,len(os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var)),end = "\t")
        if bad:
            print()
# check("notide-40")
print("full-20")
check("full-20")
print()
print("notide-20")
check("notide-20")
print()
print("notide-40")
check("notide-40")
print()
print("full-40")
check("full-40")

full-20

notide-20

notide-40
output010:	ahh 1	

full-40


In [9]:
def fix(expt,output):
    base = Path("/g/data/nm03/ab8992/outputs")
    vars = ["ahh","v","u","e","rho"]
    # vars = ["v","u","e","rho"]
    
    for var in vars:
        # data = xr.open_mfdataset(f"/g/data/nm03/ab8992/outputs/{expt}/" + output + "/" + var + "/*.nc")
        if not os.path.exists(base / expt / output / var/"backup"):
            subprocess.run(f"mkdir {base / expt / output / var}/backup",shell = True)
            subprocess.run(f"mv {base / expt / output /var}/*.nc {base / expt / output / var }/backup",shell = True)
        print(var)
        data = xr.open_mfdataset(str(base / expt / output / var /"backup") + "/*.nc",chunks = {"zl":50})
        tt.save_chunked(data,var,6,base / expt / output )

# fix("notide-40","output010")
# fix("full-20","output109")
# fix("full-20","output141")
# fix("full-40","output001")

# fix("notide-20","output015")
# fix("notide-20","output010")
# fix("notide-20","output026")
# fix("notide-20","output027")
# fix("notide-20","output025")
fix("notide-40","output010")


v
u
e
rho


In [9]:
def checkfix(expt):
    if "40" in expt:
        no = 34
    else:
        no = 17

    dirs = os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}")
    vars = ["ahh","v","u","e","rho"]
    # tell me how many files in each directory
    # print(dirs)
    for dir in dirs:
        if dir[0] != "o":
            continue

        bad = False
        for var in vars:
            # print(len(os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var)))
            
            if os.path.exists(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var) and len(os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var)) != no:
                if bad == False:
                    print(dir + ":",end = "\t")
                bad = True
                print(var,len(os.listdir(f"/g/data/nm03/ab8992/outputs/{expt}/" + dir + "/" + var)),end = "\t")
        if bad:
            print()

<xarray.Dataset>
Dimensions:  (zl: 100, time: 120, xb: 751, yb: 201)
Coordinates:
  * zl       (zl) float64 2.704 8.126 13.58 ... 5.236e+03 5.339e+03 5.443e+03
  * time     (time) object 2013-03-21 01:00:00 ... 2013-03-26 00:00:00
  * xb       (xb) float64 -0.0 2.0 4.0 6.0 ... 1.496e+03 1.498e+03 1.5e+03
  * yb       (yb) float64 -200.0 -198.0 -196.0 -194.0 ... 196.0 198.0 200.0
    lon      (yb, xb) float64 dask.array<chunksize=(1, 751), meta=np.ndarray>
    lat      (yb, xb) float64 dask.array<chunksize=(1, 751), meta=np.ndarray>
Data variables:
    ahh      (time, zl, yb, xb) float32 dask.array<chunksize=(120, 100, 1, 751), meta=np.ndarray>

## Check for 20th degree

20th is fine bc the default chunking matches 20th

In [18]:
dirs = os.listdir("/g/data/nm03/ab8992/outputs/full-20")
vars = ["ahh","v","u","e","rho"]
# tell me how many files in each directory

for dir in dirs:
    if dir[0] != "o":
        continue
    # print(dir)
    for var in vars:
        
        if os.path.exists("/g/data/nm03/ab8992/outputs/full-20/" + dir + "/" + var) and len(os.listdir("/g/data/nm03/ab8992/outputs/full-20/" + dir + "/" + var)) != 17:
            print(dir)
            print(var,len(os.listdir("/g/data/nm03/ab8992/outputs/full-20/" + dir + "/" + var)),end = "\t")
    print()





























































output065
ahh 101	output065
v 101	output065
u 101	output065
e 101	output065
rho 101	

















































output148
u 1	



output145
u 1	















output141
ahh 101	output141
v 101	output141
u 101	output141
e 101	output141
rho 101	

















output109
ahh 101	output109
v 101	output109
u 101	output109
e 101	output109
rho 101	


















In [8]:
experiment,outputs = "full-40","output*"

rawdata = tt.collect_data(
    experiment,
    outputs=outputs,
    rawdata = ["u","v"],
    chunks = {"time": -1,"xb":-1,"zl":10}
    )

ValueError: Resulting object does not have monotonic global indexes along dimension yb

In [7]:
save_vorticity("full-20","output*",recompute = True)

ValueError: Resulting object does not have monotonic global indexes along dimension yb